In [10]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [23]:
# Extract sample number using regex and sort by sample number
def extract_sample_number(file_name):
    match = re.search(r'sample #(\d+)', file_name)
    if match:
        return int(match.group(1))
    # Return a large number to push files without sample numbers to the end
    return float('inf')

def calcular_matriz_P(data_matrix, fs, ancho_banda):
    # Dimensiones de la matriz de datos (n_señales x n_muestras)
    n_signals, n_samples = data_matrix.shape

    # Calculamos los parámetros para Welch
    n_fft = int(fs / ancho_banda)  # n_fft = 20e6 / 200e3 = 100

    # Inicializamos la matriz P: (n_signals x número de bandas)
    num_bandas = n_fft // 2 + 1  # Solo mitad positiva del espectro
    P = np.zeros((n_signals, num_bandas))

    # Calculamos el espectro para cada señal (fila de data_matrix)
    for i in range(n_signals):
        f, Pxx = scipy.signal.welch(
            data_matrix[i, :], 
            fs=fs, 
            nperseg=n_fft,  # Segmentación por 100 puntos
            nfft=n_fft         # Asegura el ancho de 200 kHz por banda
        )
        P[i, :] = Pxx  # Guardamos en la matriz P


In [21]:

try:
    path_file_folder = r'C:\Samples-Hack-RF'

    # List all files in the folder
    file_list = os.listdir(path_file_folder)

    # Sort files by sample number
    sorted_file_list = sorted(file_list, key=extract_sample_number)

    # Initialize data matrix based on the sorted file list
    exploration_path = os.path.join(path_file_folder, sorted_file_list[0])
    data_exploration = np.load(exploration_path)
    
    # Aquí cambiamos a complex128 para manejar los valores complejos
    data_matrix = np.zeros((len(sorted_file_list), data_exploration.shape[0]), dtype=np.complex128)

    # Load all files into data_matrix
    for pos, file_name in enumerate(sorted_file_list):
        full_path = os.path.join(path_file_folder, file_name)
        data = np.load(full_path)
        data_matrix[pos] = data

    print("Data matrix loading complete.")

except FileNotFoundError as fnf_error:
    print(f"Error: {fnf_error}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Data matrix loading complete.


In [24]:
fs = 20e6  # Frecuencia de muestreo (ajustar a la frecuencia de muestreo de tu SDR)
ancho_banda = 200e3  # Ancho de banda de 200 kHz

# Llamada a la función con tu matriz de datos
f, Psd_m = calcular_matriz_P(data_matrix, fs, ancho_banda)

print(f"Frecuencias: {f}")
print(f"Matriz PSD (M observaciones x Canales x Frecuencias): {Psd_m.shape}")


c:\Users\david\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\signal\_spectral_py.py:462: UserWarning: Input data is complex, switching to return_onesided=False
  freqs, Pxx = csd(x, x, fs=fs, window=window, nperseg=nperseg,


ValueError: could not broadcast input array from shape (100,) into shape (51,)